In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 32.7 MB/s eta 0:00:00


In [8]:
#baseline model for aps 360 project using landmark detection

#create dataset
import mediapipe as mp
import pickle
import cv2
import os
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode = True, min_detection_confidence = 0.3)

DATA_DIR = '/content/drive/MyDrive/Subset'#change later

data = []
labels = []

for dir in os.listdir(DATA_DIR):
  for img_path in os.listdir(os.path.join(DATA_DIR, dir)): #writing the directory name into the image file name

    #create an array that stores the x and y coords of each landmark in each img
    data_aux = [] #each image gets its own array storing the x and y coords of the landmarks in that img

    img = cv2.imread(os.path.join(DATA_DIR, dir, img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#images must be rgb to work with media pipe, gotta convert from bgr to rgb, REMOVE LATER

    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:#if a hand is detected in an image
      for hand_landmarks in results.multi_hand_landmarks:
        for i in range(len(hand_landmarks.landmark)):

          #storing landmark coordinates in the array
          x = hand_landmarks.landmark[i].x
          y = hand_landmarks.landmark[i].y
          data_aux.append(x)
          data_aux.append(y)

      data.append(data_aux) #append the array of landmarks into the data array
      labels.append(dir) #CHANGE but basically the foldername is the label aka all A images in are in a folder called A


#save the data

data_path = '/content/drive/MyDrive/data.pickle'
with open(data_path, 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)
f.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#train classifier, we are going to use a random forest classifier on the landmarks

import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences

data_path = '/content/drive/MyDrive/data.pickle'
data_dict = pickle.load(open(data_path, 'rb')) #read the stored data into a dic

#basically restoring the data and labels from their stored form in a dict
data = data_dict['data']
labels = data_dict['labels']

# Pad sequences to ensure each sequence has the same length
max_seq_length = max(len(seq) for seq in data)
data = pad_sequences(data, maxlen=max_seq_length, padding='post', dtype='float32')
data = np.asarray(data)  # Convert padded sequences to numpy array
labels = np.asarray(labels)  # Convert labels to numpy array

#split data into training and test set (80% train 20% test)
data_train, data_test, label_train, label_test = train_test_split(data, labels, test_size = 0.2, shuffle = True, stratify = labels) #split data into training and test sets

#literally the model lmao
model = RandomForestClassifier()
model.fit(data_train, label_train)
predictions = model.predict(data_test)

accuracy = accuracy_score(predictions, label_test)
print('{}% of samples were classified correctly.'.format(accuracy*100))


89.04290429042905% of samples were classified correctly.
